<a href="https://colab.research.google.com/github/xrjiang83/learn_d2l/blob/master/Learn_D2L_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!nvidia-smi

Mon Nov  4 12:22:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    74W / 149W |    324MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [24]:
!pip install mxnet-cu100

In [25]:
!pip install mxnet

In [0]:
import mxnet as mx


In [0]:
import mxnet as mx
from mxnet.gluon import data as gdata

def load_data_fashion_mnist(batch_size, resize=None, root='~/.mxnet/datasets/fashion-mnist'):
    """Download the fashion mnist dataset and then load into memory."""
    #root = os.path.expanduser(root)
    transformer = []
    if resize:
      transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.FashionMNIST(train=True)
    mnist_test = gdata.vision.FashionMNIST(train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 2

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

In [0]:
# 计算分类准确率
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn
import time

def evaluate_accuracy(data_iter, net, ctx) : 
  acc_sum, n = nd.array([0], ctx=ctx), 0
  for X, y in data_iter : 
    # 如果ctx代表GPU及相应的显存，将数据复制到显存上
    X, y = X.as_in_context(ctx), y.as_in_context(ctx).astype('float32')
    acc_sum += (net(X).argmax(axis=1) == y).sum().asscalar()
    n += y.size
  return acc_sum.asscalar() / n

In [0]:
# 训练模型
def train_ch5(net, train_iter, test_iter, num_epochs, batch_size, trainer, ctx) : 
  print('training on', ctx)
  loss = gloss.SoftmaxCrossEntropyLoss()
  for epoch in range(num_epochs) : 
    train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
    for X, y in train_iter :
      X, y = X.as_in_context(ctx), y.as_in_context(ctx) 
      with autograd.record() : 
        y_hat = net(X)
        l = loss(y_hat, y).sum()
      l.backward()
      trainer.step(batch_size)
      y = y.astype('float32')
      train_l_sum += l.asscalar()
      train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
      n += y.size
    test_acc = evaluate_accuracy(test_iter, net, ctx)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec' % (epoch+1, train_l_sum / n, train_acc_sum / n, test_acc, time.time() - start))

In [0]:
#####################################################
##### Sect. 5.5 Below ###########################
#####################################################

In [31]:
def try_gpu() : 
  try: 
    ctx = mx.gpu()
    _ = nd.zeros((1,), ctx=ctx)
  except mx.base.MXNetError: 
    ctx = mx.cpu()
  return ctx

ctx = try_gpu()
ctx

gpu(0)

In [0]:
# 通过Sequential类来实现LeNet模型
net = nn.Sequential()
net.add(nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'), 
    nn.MaxPool2D(pool_size=2, strides=2), 
    nn.Conv2D(channels=16, kernel_size=5, activation='sigmoid'), 
    nn.MaxPool2D(pool_size=2, strides=2), 
    # Dense会默认将（批量大小，通道，高度，宽度）形状的输入转换成（批量大小，通道*高度*宽度）形状的输入
    nn.Dense(120, activation='sigmoid'), 
    nn.Dense(84, activation='sigmoid'), 
    nn.Dense(10))

In [0]:
# 训练数据
import sys
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size=batch_size)

In [36]:
# 训练模型
lr, num_epochs = 0.9, 50
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, num_epochs, batch_size, trainer, ctx)

training on gpu(0)
epoch 1, loss 2.3199, train acc 0.100, test acc 0.100, time 7.2 sec
epoch 2, loss 1.9340, train acc 0.254, test acc 0.574, time 7.1 sec
epoch 3, loss 0.9674, train acc 0.613, test acc 0.707, time 7.4 sec
epoch 4, loss 0.7433, train acc 0.710, test acc 0.735, time 7.3 sec
epoch 5, loss 0.6567, train acc 0.740, test acc 0.771, time 7.0 sec
epoch 6, loss 0.5970, train acc 0.764, test acc 0.776, time 7.0 sec
epoch 7, loss 0.5575, train acc 0.782, test acc 0.798, time 7.3 sec
epoch 8, loss 0.5215, train acc 0.795, test acc 0.808, time 6.9 sec
epoch 9, loss 0.4907, train acc 0.808, test acc 0.824, time 7.1 sec
epoch 10, loss 0.4704, train acc 0.820, test acc 0.828, time 7.3 sec
epoch 11, loss 0.4468, train acc 0.830, test acc 0.842, time 7.1 sec
epoch 12, loss 0.4331, train acc 0.835, test acc 0.845, time 7.3 sec
epoch 13, loss 0.4163, train acc 0.843, test acc 0.838, time 7.4 sec
epoch 14, loss 0.3992, train acc 0.851, test acc 0.856, time 7.3 sec
epoch 15, loss 0.3905, t

In [0]:
# 练习：用CPU训练LeNet试试有多慢

In [0]:
###############################################################################
######## Sect. 5.6 Below ##################################################
###############################################################################

In [39]:
# 简化版AlexNet
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata, nn
import os
import sys

# 读取数据集，使用Fashion-MNIST，但是图像高和宽扩大到224
def load_data_fashion_mnist_alexnet(batch_size, resize=None, root=os.path.join('~','.mxnet','datasets','fashion-mnist')) : 
  root = os.path.expanduser(root)
  transformer = []
  if resize : 
    transformer += [gdata.vision.transforms.Resize(resize)]
  transformer += [gdata.vision.transforms.ToTensor()]
  transformer = gdata.vision.transforms.Compose(transformer)
  mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
  mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
  num_workers = 0 if sys.platform.startswith('win32') else 4
  train_iter = gdata.DataLoader(mnist_train.transform_first(transformer), batch_size, shuffle=True, num_workers=num_workers)
  test_iter = gdata.DataLoader(mnist_test.transform_first(transformer), batch_size, shuffle=True, num_workers=num_workers)
  return train_iter, test_iter

batch_size = 128
# 如果出现了内存不足，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist_alexnet(batch_size, resize=224)

# 定义AlexNet（简化版）
net = nn.Sequential()
# 使用较大的11x11窗口来捕获物体，同时使用步幅4来较大幅度减小输出高和宽。输出通道数也比LeNet大很多
net.add(nn.Conv2D(channels=96, kernel_size=11, strides=4, activation='relu'), 
    nn.MaxPool2D(pool_size=3, strides=2), 
    # 减小卷积窗口，使用填充2来使得输入和输出的高和宽一致，且增大通道数
    nn.Conv2D(channels=256, kernel_size=5, padding=2, activation='relu'), 
    nn.MaxPool2D(pool_size=3, strides=2), 
    # 连续三个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数
    # 前两个卷积层不使用池化层来减小输入的高和宽
    nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'), 
    nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'), 
    nn.Conv2D(channels=256, kernel_size=3, padding=1, activation='relu'), 
    nn.MaxPool2D(pool_size=3, strides=2), 
    # 这里全连接层的输出个数比LeNet大数倍。使用丢弃层来缓解过拟合
    nn.Dense(4096, activation='relu'), nn.Dropout(0.5), 
    nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
    # 输出层。犹豫这里使用的是Fashion-MNIST，所以类别数为10，而非论文中的1000
    nn.Dense(10))

# 训练模型
lr, num_epochs, ctx = 0.01, 5, try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, num_epochs, batch_size, trainer, ctx)

training on gpu(0)
epoch 1, loss 1.3449, train acc 0.504, test acc 0.757, time 102.5 sec
epoch 2, loss 0.6574, train acc 0.755, test acc 0.804, time 95.1 sec
epoch 3, loss 0.5407, train acc 0.799, test acc 0.832, time 95.1 sec
epoch 4, loss 0.4728, train acc 0.826, test acc 0.849, time 95.2 sec
epoch 5, loss 0.4318, train acc 0.842, test acc 0.866, time 95.2 sec


In [0]:
i